# One Shot

Ziel ist es, mit einem Request alle Antworten zu erhalten.

In [10]:
import sys
sys.path.append('..')

import time
import survey.models 
from tqdm.notebook import tqdm
from survey.request import Request


# strategie laden
strategies = survey.models.StrategyModel()
strategies.load_strategies('oneshot_%')
strategies_list = strategies.get_strategies()

# fragen laden
questions = survey.models.QuestionsModel()
questions_list = questions.get_questions()

# models laden
models = survey.models.ModelsModel()
models_list = models.get_models()

# request und response laden
run = survey.models.RunModel()
requests = Request()
response = survey.models.ResponseModel()

In [11]:
print(f"Strategies: {len(strategies_list)} - {strategies_list}")
print(f"Questions: {len(questions_list)}")
print(f"Models: {len(models_list)}")

Strategies: 4 - [<sqlite3.Row object at 0x7f62150dc2e0>, <sqlite3.Row object at 0x7f6213069960>, <sqlite3.Row object at 0x7f621306b850>, <sqlite3.Row object at 0x7f623c29c0d0>]
Questions: 29
Models: 10


In [ ]:
for strategy in tqdm(strategies_list, desc="Strategies"):

    print(f"\n  Strategy: {strategy['name']}")

    # prompts lesen
    with open(f"../{strategy['system_path']}", "r") as f:
        system_prompt = f.read()

    with open(f"../{strategy['message_path']}", "r") as f:
        message_prompt = f.read()

    # questions
    questions_text = "\n".join([f"{id}. {text}" for id, text in questions_list])
    message = message_prompt.replace("{questions}", questions_text)


    for model_db_id, model_id in tqdm(models_list, desc=f"  Models ({strategy['name']})"):

        print(f"    {model_id}", end=" ", flush=True)

        # Run sichern
        run_id = run.write_run(model_db_id, strategy['id'])

        # Befragung
        start_time = time.time()

        answer = requests.request_model(model_id, system_prompt, message, 'COMPLETE')

        duration_time = time.time() - start_time

        # DEBUG - Was kommt zurück?
        print(f"\n=== Model: {model_id} ===")

        try:
            response.write_respone(run_id, None, answer, 'COMPLETE')
            run.update_run(answer.usage.prompt_tokens, answer.usage.completion_tokens, duration_time, run_id)
        except Exception as e:
                print(f"    --- Error {e} ---")


Strategies:   0%|          | 0/4 [00:00<?, ?it/s]


  Strategy: oneshot_none


  Models (oneshot_none):   0%|          | 0/10 [00:00<?, ?it/s]

    anthropic/claude-haiku-4-5-20251001 
=== Model: anthropic/claude-haiku-4-5-20251001 ===
    anthropic/claude-opus-4-5-20251101 
=== Model: anthropic/claude-opus-4-5-20251101 ===
    deepseek/deepseek-chat 
=== Model: deepseek/deepseek-chat ===
    deepseek/deepseek-reasoner 
=== Model: deepseek/deepseek-reasoner ===
    gemini/gemini-3-flash-preview 
=== Model: gemini/gemini-3-flash-preview ===
    gemini/gemini-3-pro-preview 
=== Model: gemini/gemini-3-pro-preview ===
    mistral/mistral-large-2512 
=== Model: mistral/mistral-large-2512 ===
    mistral/mistral-small-2506 
=== Model: mistral/mistral-small-2506 ===
    openai/gpt-5-nano-2025-08-07 
=== Model: openai/gpt-5-nano-2025-08-07 ===
    openai/gpt-5.2-2025-12-11 
=== Model: openai/gpt-5.2-2025-12-11 ===

  Strategy: oneshot_test


  Models (oneshot_test):   0%|          | 0/10 [00:00<?, ?it/s]

    anthropic/claude-haiku-4-5-20251001 
=== Model: anthropic/claude-haiku-4-5-20251001 ===
    anthropic/claude-opus-4-5-20251101 
=== Model: anthropic/claude-opus-4-5-20251101 ===
    deepseek/deepseek-chat 
=== Model: deepseek/deepseek-chat ===
    deepseek/deepseek-reasoner 
=== Model: deepseek/deepseek-reasoner ===
    gemini/gemini-3-flash-preview 
=== Model: gemini/gemini-3-flash-preview ===
    gemini/gemini-3-pro-preview 
=== Model: gemini/gemini-3-pro-preview ===
    mistral/mistral-large-2512 
=== Model: mistral/mistral-large-2512 ===
    mistral/mistral-small-2506 
=== Model: mistral/mistral-small-2506 ===
    openai/gpt-5-nano-2025-08-07 
=== Model: openai/gpt-5-nano-2025-08-07 ===
    openai/gpt-5.2-2025-12-11 
=== Model: openai/gpt-5.2-2025-12-11 ===

  Strategy: oneshot_llm_opinion


  Models (oneshot_llm_opinion):   0%|          | 0/10 [00:00<?, ?it/s]

    anthropic/claude-haiku-4-5-20251001 
=== Model: anthropic/claude-haiku-4-5-20251001 ===
    anthropic/claude-opus-4-5-20251101 
=== Model: anthropic/claude-opus-4-5-20251101 ===
    deepseek/deepseek-chat 
=== Model: deepseek/deepseek-chat ===
    deepseek/deepseek-reasoner 
=== Model: deepseek/deepseek-reasoner ===
    gemini/gemini-3-flash-preview 
=== Model: gemini/gemini-3-flash-preview ===
    gemini/gemini-3-pro-preview 
=== Model: gemini/gemini-3-pro-preview ===
    mistral/mistral-large-2512 
=== Model: mistral/mistral-large-2512 ===
    mistral/mistral-small-2506 
=== Model: mistral/mistral-small-2506 ===
    openai/gpt-5-nano-2025-08-07 
=== Model: openai/gpt-5-nano-2025-08-07 ===
    openai/gpt-5.2-2025-12-11 
=== Model: openai/gpt-5.2-2025-12-11 ===

  Strategy: oneshot_llm_explicit


  Models (oneshot_llm_explicit):   0%|          | 0/10 [00:00<?, ?it/s]

    anthropic/claude-haiku-4-5-20251001 
=== Model: anthropic/claude-haiku-4-5-20251001 ===
    anthropic/claude-opus-4-5-20251101 
=== Model: anthropic/claude-opus-4-5-20251101 ===
    deepseek/deepseek-chat 
=== Model: deepseek/deepseek-chat ===
    deepseek/deepseek-reasoner 
=== Model: deepseek/deepseek-reasoner ===
    gemini/gemini-3-flash-preview 
=== Model: gemini/gemini-3-flash-preview ===
    gemini/gemini-3-pro-preview 
=== Model: gemini/gemini-3-pro-preview ===
    mistral/mistral-large-2512 
=== Model: mistral/mistral-large-2512 ===
    mistral/mistral-small-2506 
=== Model: mistral/mistral-small-2506 ===
    openai/gpt-5-nano-2025-08-07 
=== Model: openai/gpt-5-nano-2025-08-07 ===
    openai/gpt-5.2-2025-12-11 
=== Model: openai/gpt-5.2-2025-12-11 ===
